In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as LA
%matplotlib inline

# Set custom colors
colors = [['#cf4c34','#078752','#aaaaaa','#6333ed','#d6ca54','#0ca0ad','#2ea62a','#c96b0d','#d947bb'],\
          ['#4C1C13','#073824','#555555','#3712A0','#000000','#000000','#000000','#000000','#000000'],\
          ['#FF745A','#0BD480','#cccccc','#8D66FF','#EEEEEE','#EEEEEE','#EEEEEE','#EEEEEE','#EEEEEE']]

# Set plot fonts
plt.rcParams['font.sans-serif'] = "Gill Sans MT"
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.size'] = 12
plt.rcParams['font.weight'] = 100

In [2]:
launchpadBasePosition = np.array([26.51615, 17.572+1.5, 20.04752])
launchpadOffA = np.array([-2.4,0.0,0.0])
launchpadOffB = np.array([0.0,0.0,2.4])

goalshelfBasePosition = np.array([26.57, 20.41, 1.93])
goalshelfOffA = np.array([-2.26,0.0,0.0])
goalshelfOffB = np.array([0.0, 1.70,0.0])

def goalshelfPos(shelfIndexA,shelfIndexB):
    return(goalshelfBasePosition + goalshelfOffA*shelfIndexA + goalshelfOffB*shelfIndexB )

def launchpadPos(launchpadIndexA,launchpadIndexB):
    return(launchpadBasePosition + launchpadOffA*launchpadIndexA + launchpadOffB*launchpadIndexB )

In [3]:
DRONE_SPEED = 0.018*50 # Speed in units per second (instead of units per frame)
# DRONE_SPEED = 0.018
# DRONE_SPEED = 1.8
TAKEOFF_HOVERING_TIME = 1.5/DRONE_SPEED

class Flight:
    """Class containing all the information necessary for one drone trip from the
    hover point above the launchpad to the goalpoint on the shelf.
    """
    
    def __init__(self,start_time,startpoint_indices,endpoint_indices):
        self.speed = DRONE_SPEED
        self.endpoint = goalshelfPos(endpoint_indices[0],endpoint_indices[1])
        self.startpoint = launchpadPos(startpoint_indices[0],startpoint_indices[1])
        self.start_time = start_time
        
        self.followedBy = False
        self.eventID = -1
        
        self.collidesWith = -1
        self.collidesAt = -1
        
        self.endIndices = endpoint_indices
        self.startIndices = startpoint_indices
        
        distance = self.endpoint-self.startpoint
        #print(distance)
        self.flight_vector = (distance/LA.norm(distance))*self.speed
        # calculate the position the flight would be at at time zero
        # rather than its unique start time
        self.zeroPosition = self.startpoint - start_time*self.flight_vector
        
    def position(self,time):
        """
        Returns the position the drone is at along the flightpath at the
        given time.
        
         Args:
            time (float): The query time for checking where the drone is

        Returns:
            flightposition (numpy.array): 
        """
        flightposition = time*self.flight_vector + self.zeroPosition
        return(flightposition)
    
    def closestDistanceTo(self,other):
        # the difference between the two lines:
        #     w_self(t) = a*t + b
        #     w_other(t) = c*t + d
        # (where a,b,c,d are vectors in 3-space, and t is the time parameter) will be :
        #     w_diff(t) = w_self(t) - w_other(t) = (a-c)*t + (b-d)
        # we want to find when this difference distance minimizes the two-norm:
        #     min_t \|w_diff(t) \|^2 = w_diff^T * w_diff
        # the critical point will be when the derivative of the objective is zero:
        #     0 = (a-c)^T * w_diff(t) = (a-c)^T (a-c) * t + (a-c)^T (b-d)
        # which solves for the time parameter t as:
        #     t = -<a-c, b-d> / \|a-c\|^2
        difference_flight_vector = self.flight_vector - other.flight_vector
        difference_zeroPosition = self.zeroPosition - other.zeroPosition
        differenceMagnitude = np.dot(difference_flight_vector,difference_flight_vector)
        # if the two flights are parallel than all points are equally close
        if(differenceMagnitude == 0):
            closestTime = 0
            # the difference in the flight vectors will be zero for parallel lines
            # which would cause divide by zero issues in finding a unique closest time
        else:
            closestTime = -np.dot(difference_flight_vector,difference_zeroPosition) / differenceMagnitude
        # Don't count approach times past the time limit
        if(closestTime > 180):
            closestTime = 180
        return(self.position(closestTime) - other.position(closestTime), closestTime)
    
    def endTime(self):
        distance = self.endpoint - self.startpoint
        return(self.start_time + LA.norm(distance)/self.speed + TAKEOFF_HOVERING_TIME)
    
    def __repr__(self):
        return repr((self.startIndices,self.endIndices))
    

In [4]:
COLLISION_RADIUS = 1.4

In [8]:
def generateAcceptablePath(nextTakeoff,drones,collision_tally):
    takingOffDrone = nextTakeoff.drone
    otherDrones = [drone for drone in drones if drone!=takingOffDrone]
    # Generate an approved flight
    approval = False
    path_collides = 0
    approval_attempts = 0
    while approval == False:
        newFlight = Flight(nextTakeoff.time, [random.randint(0,9),random.randint(0,2)],[random.randint(0,9),random.randint(0,3)] )
        newFlight.drone = takingOffDrone
        approval = True
        # Check for collision
        path_collides = 0
        for drone in otherDrones:
            if(drone.flightplan):
                theirFlight = drone.flightplan[-1]
                approach, approachtime = theirFlight.closestDistanceTo(newFlight)
                #print(LA.norm(approach))
                if(LA.norm(approach) < COLLISION_RADIUS):
                    if(collision_tally < min(newFlight.endTime(),180)/(10)): # enforce that the number of collisions that have occured by the n-tieth second is less than n
                        if(path_collides==0 and theirFlight.collidesWith==-1): # enforce that each path never collides with more than one other path
                            path_collides = 1
                            newFlight.collidesWith = theirFlight
                            approval = True
                        else:
                            approval = False
                    else:
                        approval = False
        approval_attempts += 1
        assert(approval_attempts < 2000),"Couldn't plan safely"
    assert(path_collides < 2)
    if(path_collides == 1):
        theirFlight = newFlight.collidesWith
        approach, approachtime = theirFlight.closestDistanceTo(newFlight)
        
        newFlight.collidesAt = newFlight.position(approachtime)
        newFlight.collidesAtTime = approachtime
        
        theirFlight.collidesWith = newFlight
        theirFlight.collidesAt = theirFlight.position(approachtime)
        theirFlight.collidesAtTime = approachtime
    return(newFlight,path_collides)

In [9]:
class Drone:
    def __init__(self,name,number):
        self.flightplan = []
        self.name = name
        self.IDnumber = number
        
    def newTakeoff(self,flight):
        self.flightplan.append(flight)
        
    def __repr__(self):
        return repr((self.name))
    
    def position(self, time):
        currentFlight = next(flight for flight in self.flightplan if flight.endTime()>time)
        return( currentFlight.position(time) )
    
    def currentFlight(self,time):
        currentFlight = next(flight for flight in self.flightplan if flight.endTime()>time)
        return(currentFlight)

In [ ]:
import random
NUM_DRONES = 10

class Takeoff:
    def __init__(self,time,drone):
        self.time = time
        self.drone = drone
        
    def __repr__(self):
        return repr((self.time, self.drone))

collision_tally = 0
while collision_tally < 20:

    drones = []
    upcomingTakeoffs = []
    collision_tally = 0
    curEventID = 0

    for ii in range(NUM_DRONES):
        newDrone = Drone("Drone #"+str(ii),ii)
        drones.append(newDrone)
        launch = Takeoff(0.0+ii*10.0/NUM_DRONES,newDrone)
        upcomingTakeoffs.append( launch )
        #newFlight, path_collides = generateAcceptablePath(launch,drones,collision_tally)

    #print("initial takeoff created "+str(collision_tally)+" collisions")

    # Pick out the first respawn takeoff
    upcomingTakeoffs = sorted(upcomingTakeoffs, key=lambda takeoff: takeoff.time)
    nextTakeoff = upcomingTakeoffs.pop(0)
    takeoffOrderedFlights = []

    while nextTakeoff.time < 180:
        takingOffDrone = nextTakeoff.drone
        newFlight, path_collides = generateAcceptablePath(nextTakeoff,drones,collision_tally)
        collision_tally += path_collides
        # Record in takeoff history
        newFlight.eventID = curEventID
        curEventID += 1
        takeoffOrderedFlights.append(newFlight)
        # Assign this approved flight to the drone
        if(takingOffDrone.flightplan):
            takingOffDrone.flightplan[-1].followedBy = newFlight
        takingOffDrone.newTakeoff(newFlight)
        # Queue up the following takeoff once this new flight terminates
        upcomingTakeoffs.append(Takeoff(newFlight.endTime(),takingOffDrone))
        # Sort the takeoffs to identify who is taking off next
        upcomingTakeoffs = sorted(upcomingTakeoffs, key=lambda takeoff: takeoff.time)
        nextTakeoff = upcomingTakeoffs.pop(0)

print("done with "+str(collision_tally)+" collisions")
# Show me the flight plan for a drone
drones[0].flightplan

for flight in takeoffOrderedFlights:
    collidesWithReport = ""
    if(flight.collidesWith == -1):
        collidesWithReport = [-2,-2]
    else:
        print(flight.eventID, flight.drone.IDnumber,
              flight.collidesWith.drone.IDnumber,flight.collidesWith.eventID,flight.collidesAt)

In [14]:
import csv
with open('flightplan.csv', 'w') as csvfile:
    planwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    planwriter.writerow(['eventID', 'droneID', 'startingLaunchpadIndexX', 'startingLaunchpadIndexY',
                         'shelfIndexX', 'shelfIndexY', 'followedBy',
                         'teleportToLaunchpadIndexX', 'teleportToLaunchpadIndexY', 'collidesWithNum', 'collidesAtPos', 'collidesAtTime', 'collidesWithEvent'])
    for flight in takeoffOrderedFlights:
        collidesWithReport = ""
        if(flight.collidesWith == -1):
            collidesWithReport = [-2,-2,-2,-2]
        else:
            collidesWithReport = [flight.collidesWith.drone.IDnumber,flight.collidesAt,flight.collidesAtTime,flight.collidesWith.eventID]
        if(flight.followedBy):
            planwriter.writerow([flight.eventID, flight.drone.IDnumber] + flight.startIndices + flight.endIndices
                                + [flight.followedBy.eventID] + flight.followedBy.startIndices
                                + collidesWithReport)
        else:
            planwriter.writerow([flight.eventID, flight.drone.IDnumber] + flight.startIndices + flight.endIndices
                                + [-1] + [-1,-1]
                                + collidesWithReport)

# Preview what the Flightplan looks like

In [ ]:
# Preview what the CSV exports

for flight in takeoffOrderedFlights:
    collidesWithReport = ""
    if(flight.collidesWith == -1):
        collidesWithReport = [-2,-2]
    else:
        collidesWithReport = [flight.collidesWith.drone.IDnumber,flight.collidesAt]
    if(flight.followedBy):
        print([flight.eventID, flight.drone.IDnumber] + flight.startIndices + flight.endIndices
                            + [flight.followedBy.eventID] + flight.followedBy.startIndices
                            + collidesWithReport)
    else:
        print([flight.eventID, flight.drone.IDnumber] + flight.startIndices + flight.endIndices
                            + [-1] + [-1,-1]
                            + collidesWithReport)

In [ ]:
# Explore particular collision events of interest (set the IDs to whatever you like)
flight = takeoffOrderedFlights[36]
print(flight.eventID, flight.drone.IDnumber,
              flight.collidesWith.drone.IDnumber,flight.collidesWith.eventID,flight.collidesAt)
flight = takeoffOrderedFlights[36].collidesWith
print(flight.eventID, flight.drone.IDnumber,
              flight.collidesWith.drone.IDnumber,flight.collidesWith.eventID,flight.collidesAt)
flight = takeoffOrderedFlights[37]
print(flight.eventID, flight.drone.IDnumber, flight.startpoint, flight.endpoint)

In [ ]:
# Calculate the average flight time

def goalshelfPos(shelfIndexA,shelfIndexB):
    return(goalshelfBasePosition + goalshelfOffA*shelfIndexA + goalshelfOffB*shelfIndexB )

def launchpadPos(launchpadIndexA,launchpadIndexB):
    return(launchpadBasePosition + launchpadOffA*launchpadIndexA + launchpadOffB*launchpadIndexB )

total_distances = 0.0
total_arcs = 0.0
for startX in range(0,9):
    for startY in range(0,2):
        for endX in range(0,9):
            for endY in range(0,3):
                total_arcs += 1.0
                total_distances += LA.norm(goalshelfPos(startX,startY) - launchpadPos(endX,endY))

average_trip_distance = total_distances/total_arcs
average_trip_time = average_trip_distance/DRONE_SPEED
print(average_trip_time)

In [ ]:
# Animate the trajectories
from matplotlib import animation, rc # import the animation libraries
from IPython.display import HTML # import libraries to embed animation in Jupyter
%matplotlib inline

framerate = 10
timestep = 1/framerate

# Create the figure
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-2, 30), ylim=(-2, 30))
#ax.grid()

# Initialize the lines for drawing the cars
for drone in drones:
    line = ax.plot([], [], '-', lw=2, color = colors[0][1])
    drone.plot = line
    drone.text = ax.text(0.05, 0.9, drone.IDnumber, transform=ax.transAxes)
# Labeling
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

# Convenience function for plotting rectangular perimeter of a 4 meter by 2 meter car
def drawUAV(line,position):
    centerX = position[0]
    #closeness = 0.4+0.8*(position[0]-19.0)/(27.0-19.0)
    centerZ = position[2]
    #xpoints = [centerX-1,centerX-1,centerX+1,centerX+1,centerX-1]
    #ypoints = [centerZ+1,centerZ-1,centerZ-1,centerZ+1,centerZ+1]
    xpoints = centerX + np.cos(np.arange(0,6.28,0.2))*COLLISION_RADIUS/2
    ypoints = centerZ + np.sin(np.arange(0,6.28,0.2))*COLLISION_RADIUS/2
    line.set_data(xpoints,ypoints)

# Function for intializing the animation
def init():
    for drone in drones:
        drone.plot[0].set_data([], [])
        drone.text.set_text(drone.IDnumber)
    time_text.set_text('')
    return drones[0].plot

# Animation function called each time step
def animate(ii):
    # Draw the cars at this time step
    for drone in drones:
        drawUAV(drone.plot[0],drone.position(ii))
        #screenpos = ax.transData.transform((drone.position(ii)[0],drone.position(ii)[2]))
        drone.text.set_y((drone.position(ii)[2]+2)/32)
        drone.text.set_x((drone.position(ii)[0]+2)/32)
        #drone.text.set_y(screenpos[1])
        # Change the color for collisions
        currentFlight = next(flight for flight in drone.flightplan if flight.endTime()>ii)
        if(currentFlight.collidesWith != -1):
            if(currentFlight.collidesAtTime - 4 < ii and ii < currentFlight.collidesAtTime + 4):
                drone.plot[0].set_color(colors[1][0])
            else:
                drone.plot[0].set_color(colors[0][1])
        else:
            drone.plot[0].set_color(colors[0][1])
    # Update the time label
    time_text.set_text(time_template % (ii))
    
    return drones[0].plot

# Generate the animation
anim = animation.FuncAnimation(fig, animate, np.arange(0,180,timestep),
                              interval=100, blit=True, init_func=init)
# Save the animation to a file (with 15 frames per second)
#mywriter = animation.FFMpegWriter(fps=15) # increase number for faster animation
mywriter = animation.FFMpegWriter(fps=framerate)
anim.save('flightplan.mp4', writer=mywriter)
HTML(anim.to_html5_video())
rc('animation', html='html5')
#anim
#plt.show()

In [ ]:
approach, approachtime = takeoffOrderedFlights[36].closestDistanceTo(takeoffOrderedFlights[37])
LA.norm(approach)